# 第二课 数字冰壶比赛中的Socket通信

## 2.1 Socket通信

Socket是一种抽象层，通常翻译为套接字，应用程序通过它来发送和接收数据，使用Socket可以将应用程序添加到网络中，与处于同一网络中的其他应用程序进行通信。

Socket可以理解为是两个程序进行双向数据传输的网络通信的端点，一般由一个IP地址加上一个端口号来表示。每个程序都在一个端口上提供服务，而想要使用该服务的程序则需要连接该端口。

这就好比是在两个程序之间搭建了一根管道，程序A可以通过管道向程序B发送数据，程序B可以接受管道传输来的数据，同样程序B也可以发送，这个管道就相当于Socket的作用。

<center><img src="img/Socket.png" width=600px></center>

根据不同的的底层协议，Socket通信的实现是多样化的，最常用到的是基于TCP/IP协议族的Socket通信，而在这个协议族当中主要的Socket类型为流套接字（streamsocket）和数据报套接字(datagramsocket)。

流套接字将TCP协议作为其端对端协议，提供了一个可信赖的字节流服务，是有连接的通信方式，通信双方在开始时必须建立一次连接过程，建立一条通信链路。数据报套接字将UDP协议作为其端对端协议，提供数据打包发送服务，是无连接的通信方式，通信双方不存在一个连接过程，一次网络I/O以一个数据报形式进行。

在数字冰壶比赛中，用到的就是使用TCP协议的有连接的流套接字通信。

有连接Socket通信和打电话很相似。你要打电话给一个朋友，一般要经历下面三个步骤：

1. 拨号，对方听到电话铃声后提起电话，这时双方就建立起了连接；
2. 双方通话；
3. 挂断电话结束此次交谈。

类似地，有连接Socket通信要经历下面三个步骤：

1. 创建Socket，双方建立连接；
3. 双方按照一定的协议对Socket进行读写操作；
3. 关闭Socket。

## 2.2 启动数字冰壶比赛服务器

想要给朋友打电话，首先要有一个朋友。要编写程序和数字冰壶比赛服务器通讯，当然也要先启动数字冰壶比赛服务器。

点击页面左上角Jupyter菜单中的[Run]菜单项，点击该该菜单项的[Start Curling Server]子菜单项，即可启动一个数字冰壶比赛服务器。

<center><img src="img/StartServer.png" width=600px></center>

数字冰壶比赛服务器会在新开浏览器页面中启动，第一次加载速度较慢，请耐心等待。

数字冰壶比赛服务器主界面如下图所示，请注意界面右下角给出的连接信息，<b>每次启动服务器该信息中的具体数据都会有变化</b>，这些数据在后续的程序代码编写中会用到。

<center><img src="img/CurlingServer.png"></center>

## 2.3 编写AI选手实现和服务器的Socket通信

### 2.3.1 创建Socket连接

因为要操作Socket，所以需要用到Socket模块，python 內建的模块就很方便，运行导入模块的范例代码如下所示：

> 本课程采用jupyter平台实现在线编程教学，给出的所有代码单元都可以在教程页面中直接运行，运行方式有以下两种：<br>
>> 1. 点击代码单元放置光标在任意位置，然后点击课程页面上方工具条中的三角形按钮，会运行该段代码然后选中下一个单元；
>> 2. 点击代码单元放置光标在任意位置，然后按下Ctrl+Enter组合键，会运行该段代码并停留在当前单元。


In [ ]:
#导入socket模块
import socket

socket类的构造函数的语法格式如下所示：

<b>s = socket.socket( [family [, type [, protocol]]] )</b><br>
参数：family - 套接字家族，AF_INET代表基于网络类型，AF_UNIX代表基于文件类型，默认为AF_INET；<br>
    　　　type - 套接字通信类型，SOCK_STREAM代表使用TCP协议的流套接字通信，SOCK_DGRAM代表使用UDP协议的数据报套接字通信，SOCK_RAW代表使用其他协议的套接字通信，默认为SOCK_STREAM。<br>
　　　protocol: type为SOCK_RAW才需要这个参数，指定使用协议，默认为 0。<br>
返回值：socket对象

在和数字冰壶比赛服务器的通信过程中，使用的是基于网络类型的流套接字通信，新建Socket对象的范例代码如下所示：

In [ ]:
#新建Socket对象
ai_sock =socket.socket()

初始化Socket对象后，即可开始创建Socket连接，在数字冰壶比赛服务器上已经创建了服务器Socket，这里只需要创建客户端Socket。相关函数的语法格式如下所示：

<b>s.connect(address)<br></b>
参数：address - 连接地址，格式为元组(host,port)，其中host为服务器IP，port为连接端口。<br>
返回值：无

根据数字冰壶服务器主界面右下角显示的连接信息，编写创建Socket连接的范例代码如下所示：

In [ ]:
#服务器IP(固定不变无需修改)
host = '192.168.0.127'
#连接端口(固定不变无需修改)
port = 7788

#创建Socket连接
ai_sock.connect((host,port))
print("已建立socket连接", host, port)

### 2.3.2 连接到数字冰壶服务器核对密钥

在数字冰壶服务器的主界面中点击【投掷调试】按钮，即可进入如下图所示的调试界面。

<center><img src="img/CurlingTest1.png"></center>

投掷调试和正式对战的流程是相同的，AI选手需要先向服务器发送连接密钥，服务器核对密钥无误后，会向AI选手发送临时选手名。

> 正式对战时，临时选手名为Player1代表首局先手，Player2代表首局后手。投掷调试时，临时选手名一律为Player1。

Socket对象发送/接收消息相关函数的语法格式如下所示：

<b>ret = s.send(data)</b><br>
参数：data - 待发送的数据，要求是bytes类型数据，字符串类型数据需要调用encode()方法编码后再行发送。<br>
返回值：发送的数据长度

<b>data = s.recv(size)</b><br>
参数：size - 接收数据的最大长度。<br>
返回值：接收到的数据，类型为bytes，字符串数据需要调用decode()方法进行解码。

向数字冰壶服务器发送密钥并从数字冰壶服务器接收临时选手名的范例代码如下所示，其中key的值需要根据如图所示数字冰壶服务器主界面右下角显示的连接信息中ConnectKey的内容进行修改：

In [ ]:
#根据数字冰壶服务器界面中给出的连接信息修改CONNECTKEY，注意这个数据每次启动都会改变。
#key = "test2023_45bfebbe-7185-4366-851d-90018fffbb6e"
key = "test2023_2_5d86e50a-d561-4dd5-94e1-10f2051210a8"

#通过socket对象发送消息
msg_send = 'CONNECTKEY:' + key  # + '\0'
ai_sock.send(msg_send.encode())

#通过socket对象接收消息
msg_recv = ai_sock.recv(1024)
print(msg_recv)

上方代码成功运行后，会收到类似于 b'CONNECTNAME Player1 \x00' 的消息数据。数据开头的b说明这个数据是bytes类型；消息代码和参数是以空格分隔开，如果有多个参数，各个参数之间也是以空格进行分隔；末尾的\x00说明是以0结尾，实际处理时需要去掉末尾的0并转换成字符串类型再进一步解析。

### 2.3.3 确认选手已准备完毕

后续过程中需要多次编码发送消息，以及多次接收消息进行解析，按照编程惯例，将这两个过程编写为函数方便调用。范例代码如下所示：

In [ ]:
import time

#通过socket对象发送消息
def send_msg(sock, msg):
    print("  >>>> " + msg)
    #将消息数据从字符串类型转换为bytes类型后发送
    sock.send(msg.strip().encode())
    
#通过socket对象接收消息并进行解析
def recv_msg(sock):
    #为避免TCP粘包问题，数字冰壶服务器发送给AI选手的每一条信息均以0（数值为0的字节）结尾
    #这里采用了逐个字节接收后拼接的方式处理信息，多条信息之间以0为信息终结符
    buffer = bytearray()
    while True:
        #接收1个字节
        data = sock.recv(1)
        #接收到空数据或者信息处终结符(0)即中断循环
        if not data or data == b'\0':
            time.sleep(0.1)
            break
        #将当前字节拼接到缓存中
        buffer.extend(data)
    #将消息数据从bytes类型转换为字符串类型后去除前后空格
    msg_str = buffer.decode().strip()
    print("<<<< " + msg_str)

    #用空格将消息字符串分隔为列表
    msg_list = msg_str.split(" ")
    #列表中第一个项为消息代码
    msg_code = msg_list[0]
    #列表中后续的项为各个参数
    msg_list.pop(0)
    #返回消息代码和消息参数列表
    return msg_code, msg_list

成功连接到数字冰壶服务器核对密钥后，数字冰壶服务器的界面上会显示＜Player1 已连接＞，如图所示。

> 在投掷调试模式中，数字冰壶服务器接收到AI选手的连接信息后，会启动一个笨笨的对手机器人，所以界面上还会显示＜Player2 已连接＞。

<center><img src="img/CurlingTest2.png"></center>

点击界面上的【准备】按钮后，数字冰壶服务器会向AI选手发送“ISREADY”消息，AI选手需要回复“READYOK”消息，表示已经准备完毕，并需要发送带有参数的“NAME”消息，将AI选手的选手名发送到数字冰壶服务器，消息代码和参数之间用空格分隔。范例代码如下所示：

In [ ]:
#接收消息并解析
msg_code, msg_list = recv_msg(ai_sock)
#如果消息代码是"ISREADY"
if msg_code == "ISREADY":
    #发送"READYOK"
    send_msg(ai_sock, "READYOK")
    #发送"NAME"和AI选手名
    send_msg(ai_sock, "NAME CurlingAI")

### 2.3.4 开始对战/投掷调试

确认AI选手已准备好，数字冰壶服务器的界面上会显示＜CurlingAI 已准备＞，如图所示。

> 在投掷调试模式中，数字冰壶服务器接收到AI选手的准备信息后，会通知笨笨的对手机器人也完成准备，所以界面上还会显示＜Robot 已准备＞。

<center><img src="img/CurlingTest3.png"></center>

点击界面上的【开始对局】按钮后，服务器会向AI选手发送“NEWGAME”消息，并会跳转到对战/投掷调试界面。

每局对战或投掷调试开始时，以及后续每一个冰壶球投掷完成后，会向AI选手发送多条消息通知当前的比赛状态和冰壶球坐标。

我们先看下如何解析比赛状态信息。比赛状态信息的消息代码是“SETSTATE”，该消息有四个参数，参数1是当前完成投掷数，参数2是当前完成对局数，参数3是总对局数，参数4是预备投掷者（0为持蓝色球者，1为持红色球者）。

接收并解析比赛状态信息的范例代码如下所示：

In [ ]:
while(1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"SETSTATE"
    if msg_code=="SETSTATE":
        print("当前完成投掷数：", msg_list[0])
        print("当前完成对局数：", msg_list[1])
        print("总对局数：", msg_list[2])
        if int(msg_list[3]) == 0:
            print("预备投掷者：蓝壶")
        else:
            print("预备投掷者：红壶")
        break

通过对接收到的比赛状态消息进行解析，可以看到比赛共有4局，当前是第1局的第1次投掷，预备投掷者是蓝壶。

在轮到当前AI选手执行投掷动作时，服务器会向当前AI选手发送“GO”消息。

在接收到服务器发来的“GO”消息后，AI选手就可以进行投壶了。投壶消息代码为“BESTSHOT”，这个消息带有三个参数，参数1是冰壶投掷时的初速度v0（0≤v0≤6），参数2是冰壶投掷时的横向偏移h0（-2.23≤h0≤2.23），参数3是冰壶投掷时的初始角速度ω0（-3.14≤ω0≤3.14），消息代码和各个参数之间均用空格分隔。

如下所示范例代码，在接收到"GO"消息后，发出带参数的"BESTSHOT"消息，投出了初速度为3、横向偏移为0、初始角速度为0的冰壶。

In [ ]:
while(1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"GO"
    if msg_code == "GO":
        print("============第1局第1壶============")
        #发送投壶消息：初速度为3；横向偏移为0，初始角速度为0
        send_msg(ai_sock, "BESTSHOT 3.1 1.9 0")
        break

服务器接收到来自AI选手的投壶消息后，即会根据参数中指定的初速度、横向偏移和初始角速度对冰壶的运动轨迹进行仿真并在界面上实时展示，如下图所示。

> 在投掷调试模式中，数字冰壶服务器接收到AI选手的投掷信息后，会通知笨笨的对手机器人投出一个出界壶，所以界面上还会展示红壶的投壶过程，但由于投出的是出界壶，所以该壶最终不会停留在界面中。

<center><img src="img/CurlingTest4.png"></center>

需要注意的是，数字冰壶服务器中对于场地各点的摩擦系数引入了随机变量，<b>即便是相同的参数进行投掷，每次的落点也会有少许的差别</b>。这样就为比赛增加了一些偶然的成分，更贴近实际的冰壶比赛，也更具有趣味性。

在实际对战中，AI选手从接收到“GO”消息开始，到发出“BESTSHOT”消息为止，间隔时间不能超过2分钟，超过2分钟就会判超时，轮到对手继续投壶。

> 在投掷调试时没有超时限值。

下面我们保持投壶的初速度和初始角速度不变，改变横向偏移为0.5，观察冰壶球的落点变化。

In [ ]:
while(1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"GO"
    if msg_code == "GO":
        print("============第1局第2壶============")
        #发送投壶消息：初速度为3；横向偏移为0.5，初始角速度为0
        send_msg(ai_sock, "BESTSHOT 3.0 0.5 0")
        break

投壶结果如下图所示，对比两个壶的位置，可以看到第2壶由于有横向偏移，和第1壶相比距离场地中线更远。

> 正的横向偏移会使得投出的冰壶向右偏移，而负的横向偏移会使得投出的冰壶向左偏移。

<center><img src="img/CurlingTest5.png"></center>

接下来我们保持投壶的初速度不变，将横向偏移恢复为0，改变初始角速度为-3.14，观察冰壶球的落点变化。

In [ ]:
while(1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"GO"
    if msg_code == "GO":
        print("============第1局第3壶============")
        #发送投壶消息：初速度为3；横向偏移为0，初始角速度为-3.14
        send_msg(ai_sock, "BESTSHOT 3.0 0 -3.14")
        break

投壶结果如下图所示，对比第3壶和第1壶的位置，可以看到第3壶也偏离了场地中线，这是因为给定了初始角速度就会投出弧线球，和原有的直线投壶落点自然会有差异。

> 正的初始角速度会投出偏转到右侧的弧线球，而负的初始角速度会投出偏转到右侧的弧线球。

<center><img src="img/CurlingTest6.png"></center>

接下来我们将投壶的初速度改为2.5，保持横向偏移为0不变，初始角速度也恢复为0，观察冰壶球的落点变化。

In [ ]:
while(1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"GO"
    if msg_code == "GO":
        print("============第1局第4壶============")
        #发送投壶消息：初速度为2.5，横向偏移为0，初始角速度为0
        send_msg(ai_sock, "BESTSHOT 2.5 0 0")
        break

投壶结果如下图所示，可以看到第4壶由于初速度较小，未能将冰壶投到大本营中，而是停留在了防守区内。

<center><img src="img/CurlingTest7.png"></center>

### 2.3.5 分析得分区局势

接下来我们看下如何对得分区的冰壶位置信息进行解析。冰壶位置信息的消息代码是“POSITION”，该消息有32个参数，分别是16个冰壶球的当前坐标，顺序同当前对局投掷顺序，（0，0）坐标表示未投掷或已出界的球。

接收并解析比赛状态信息和冰壶位置信息的范例代码如下所示：

In [ ]:
init_x, init_y, gote_x, gote_y = [0]*8, [0]*8, [0]*8, [0]*8
while(1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"POSITION"
    if msg_code=="POSITION":
        for n in range(8):
            init_x[n], init_y[n] = float(msg_list[n*4]), float(msg_list[n*4+1])
            print("先手第%d壶坐标为(%.4f, %.4f)" % (n+1, init_x[n], init_y[n]))
            gote_x[n], gote_y[n] = float(msg_list[n*4+2]), float(msg_list[n*4+3])
            print("后手第%d壶坐标为(%.4f, %.4f)" % (n+1, gote_x[n], gote_y[n]))
        break

通过对接收到的消息进行解析，可以看到先手第1壶、第2壶、第3壶和第4壶有坐标，其余的壶由于已出界或未投掷，坐标都为(0,0)。

对于有坐标的冰壶，如何判断它们是不是在大本营内呢？根据《1.3 数字冰壶平台》的内容可知，冰壶半径为0.145米，场地远端大本营圆心坐标为(2.375, 4.88)，半径为1.830米。根据这些参数，就可以算出冰壶距离大本营圆心的距离，进一步判断这个壶是否在大本营内。

范例代码如下所示：

In [ ]:
import math

#与大本营中心距离
def get_dist(x,y):
    House_x = 2.375
    House_y = 4.88
    return math.sqrt((x-House_x)**2+(y-House_y)**2)

#大本营内是否有壶
def is_in_house(dist):
    House_R = 1.830
    Stone_R = 0.145
    if dist<(House_R+Stone_R):
        return 1
    else:
        return 0

for n in range(8):
    if (init_x[n] > 0) and (init_y[n] > 0):
        distance = get_dist(init_x[n], init_y[n])
        print("先手方第%d壶距离大本营中心%.2f米" % (n+1, distance))
        if ( is_in_house(distance) ):
            print("该壶在大本营内！")
        else:
            print("该壶不在大本营内！")

### 2.3.6 获取每局得分及整场比赛得分

重复如上所述接收消息、解析消息、发送投掷命令的过程，完成本局剩余4个壶的投掷，范例代码如下：

In [ ]:
import random

#循环投掷剩余的4个壶
for n in range(4):
    while(1):
        #接收消息并解析
        msg_code, msg_list = recv_msg(ai_sock)
        #如果消息代码是"GO"
        if msg_code == "GO":
            print("============第1局第" + str(n+5) + "壶============")
            #冰壶初始速度取2.5到3.5之间的随机数
            v0 = 2.5 + random.random()
            #冰壶横向偏移取-1到1之间的随机数
            h0 = -1.0 + 2.0*random.random()
            #发送投壶消息：初速度为v0；横向偏移为h0，初始角速度为0
            send_msg(ai_sock, "BESTSHOT " + str(v0) + " " + str(h0) + " 0")
            break

每局全部八个冰壶球投掷完毕后，服务器会向AI选手发送“SCORE”消息，参数是该局得分。正分说明是自己得分，负分说明是对方得分。

忽略比赛状态消息和冰壶球坐标消息，仅处理得分的范例代码如下：

In [ ]:
while (1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"SCORE"
    if msg_code == "SCORE":
        #从消息参数列表中获取得分
        score = int(msg_list[0])
        if  score > 0:
            print("我方得"+str(score)+"分")
        elif score < 0:
            print("对方得"+str(score*-1)+"分")
        else:
            print("双方均未得分")
        break

此时界面上也会给出当前的得分情况，如下图所示。

<center><img src="img/CurlingTest8.png"></center>

点击【下一局】按钮，即会开始新的一局对局/投掷调试，初赛阶段和投掷调试模式设定为每场比赛四局，决赛阶段设定为每场比赛八局。

运行下方代码可以在随后的三局比赛中实现随机投壶。<b>注意每一局结束后都要在数字冰壶服务器界面中点击【下一局】按钮</b>。

In [ ]:
#循环后续3局比赛
for m in range(3):
    #循环投掷8个壶
    for n in range(8):
        while(1):
            #接收消息并解析
            msg_code, msg_list = recv_msg(ai_sock)
            #如果消息代码是"GO"
            if msg_code == "GO":
                print("============第" +str(m+2) + "局第" + str(n+1) + "壶============")
                #冰壶初始速度取2.5到3.5之间的随机数
                v0 = 2.5 + random.random()
                #冰壶横向偏移取-1到1之间的随机数
                h0 = -1.0 + 2.0*random.random()
                #发送投壶消息：初速度为v0；横向偏移为h0，初始角速度为0
                send_msg(ai_sock, "BESTSHOT " + str(v0) + " " + str(h0) + " 0")
                break

全部对局结束后，服务器会向AI选手发送“TOTALSCORE”消息和“GAMEOVER”消息。“TOTALSCORE”有两个参数，参数1是执蓝壶选手总得分，参数2是执红壶选手总得分；“GAMEOVER”消息有一个参数，“WIN”代表胜利，“LOSE”代表失败，“DRAW”代表平局。

In [ ]:
while (1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果消息代码是"GAMEOVER"
    if msg_code == "GAMEOVER":
        score = int()
        if  msg_list[0] == "WIN":
            print("我方获胜")
        elif msg_list[0] == "LOSE":
            print("对方获胜")
        else:
            print("双方平局")
        break

### 2.3.7 关闭Socket连接

全部对局结束后，数字冰壶服务器界面上显示的计分板也给出总比分，如下图所示。

<center><img src="img/CurlingTest9.png"></center>

点击【返回主菜单】按钮，即可退出投掷调试模式。此时服务器会向AI选手连续发送5条空信息。而AI选手在连续检测到5条空信息之后，就应该调用socket对象的close()方法关闭socket连接。范例代码如下：

> 先点击【返回主菜单】按钮再运行下方代码

In [ ]:
#空消息计数器归零
retNullTime = 0
while (1):
    #接收消息并解析
    msg_code, msg_list = recv_msg(ai_sock)
    #如果接到空消息则将计数器加一
    if msg_code == "":
        retNullTime = retNullTime + 1
    #如果接到五条空消息则关闭Socket连接
    if retNullTime == 5:
        #关闭Socket连接
        ai_sock.close()
        print("已关闭socket连接")
        break

## 小结

本课结合Socket通讯的流程，从AI选手和数字冰壶客户端建立连接开始，逐行代码介绍了如何和数字冰壶服务器的收发消息，最终在数字空间中完成了一场随机投壶的独角戏比赛。

在真正的数字冰壶比赛中，每次投壶就不能这么草率了。在投壶前需要对大本营状态进行分析，再结合比赛状态信息，综合考虑各种情况，制定对战策略，确定每一壶的投掷目标，并努力实现。

> 如何实现投掷目标？请参考《3.数字冰壶比赛中的冰壶运动模型》<br>
> 如何制定对战策略？请参考《4.数字冰壶比赛中的对战策略》